In [ ]:
# goal
goal = "Convert a long YouTube transcript into structured notes and diagrams for a local material-mkdocs site."

# tasks
task_1 = "Download the complete transcript from the YouTube video."
task_2 = "Collect video metadata (title, author, date) from YouTube."
task_3 = "Identify key topics, keywords and questions from the transcript."
task_4 = "Draft an outline with clear sections and subsections based on the transcript."
task_5 = "Populate each subsection with detailed content from the transcript."
task_6 = "Create diagrams based on the identified topics and integrate them into the outline."
task_7 = "Assemble each section into individual Markdown files, including relevant diagrams and metadata."
task_8 = "Compile all Markdown files into the material-mkdocs site structure."
task_9 = "Launch the local mkdocs site to verify the structure and content."